This notebook is based on `5.0-ayw-neural-network.ipynb`

### Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

import os
import datetime

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error

import tensorflow as tf

from src.models.prepare_model import load_PI_data, split_data_training, create_fully_connected_model
from src.models.train_model import train_model
from src.models.predict_model import load_and_predict_model

tf.random.set_seed(42)
np.random.seed(42)

### Loading and preparing the data for the neural network

In [2]:
# Relevant paths:
# '../data/processed/synthetic_basic_pi_daily.zip'
# '../data/processed/synthetic_soil_pi_daily.zip'
# '../data/processed/synthetic_weather_pi_daily.zip'
# '../data/processed/synthetic_soil_weather_pi_daily.zip'
X, y = load_PI_data('../data/processed/synthetic_basic_pi_daily.zip', 50, 5)
X_train, X_valid, X_test, y_train, y_valid, y_test = split_data_training(X, y)


In [ ]:
# Check that the data looks like what you expect,
X = X_train
y = y_train

plt.figure(figsize=(15,20))
plt.suptitle("Examples of the loaded data: input")
for i in range(0, 10):
    plt.xlabel("Time steps")
    plt.ylabel("PI")
    plt.subplot(5,2,i+1)
    plt.plot(X[i])
plt.show()

plt.figure()
plt.title("Examples of the loaded data: target")
plt.xlabel("Time steps")
plt.ylabel("Degradation factor")
for i in range(0, 10):
    plt.plot(y[i])
plt.show()

# Create and train the model

In [ ]:
# First create a model 
signal_length = 1825
model = create_fully_connected_model(signal_length, signal_length, [signal_length] * 5)
model_name = "fc_6layers"

In [ ]:
# Get run number 
run_index_path = "./checkpoint/run_index"

with open(run_index_path, 'r') as f:
    run_index = int(f.readline())
run_index += 1
with open(run_index_path, 'w') as f:
    f.write(str(run_index))
    
checpoint_path = train_model(model,
                             X_train,
                             y_train,
                             X_valid,
                             y_valid,
                             run_index,
                             model_name,
                             "./log/basic",
                             "./checkpoint/basic",
                             1000)

In [ ]:
# Loading the best parameters and generating a list of predictions.
prediction = load_and_predict_model("checkpoint/basic/fc_6layers_run001.h5", X_test)

In [ ]:
# Visualizing the predictions.
plt.figure(figsize=(10,6))
plt.tight_layout()
plt.suptitle("Degradation factor vs time step")
for i in range(0, min(len(prediction), 9)):
    plt.subplot(3,3,i+1)
    plt.ylim(0.94,1.01)
    plt.plot([x for x in prediction[i]])
    plt.plot([x for x in y_test[i]])
plt.legend(['NN prediction','Actual degradation'])
plt.show()

plt.figure(figsize=(10,3))
plt.suptitle("Degradation factor vs timestep: predicted (left) vs target (right)")
plt.subplot(1,2,1)
plt.ylim(0.90,1)
for i in range(0,len(prediction)):
    plt.plot(prediction[i])
plt.subplot(1,2,2)
plt.ylim(0.90,1)
for i in range(0,len(y_test)):
    plt.plot(y_test[i])
plt.show()

In [ ]:
# To convert a decay series into a decay rate.
def extract_yr_degrad(avg_power_decay):
    x = np.array([i/365 for i in range(0,len(avg_power_decay))])
    y = np.array(avg_power_decay)
    return np.polyfit(x,y,1)[0]

# Computing the decay rates from the prediction and target, and
# computing the RMSE between those.
model_pred = []
degradation_rates_test = []
for i in range(0, len(prediction)):
    model_pred.append(extract_yr_degrad(prediction[i, 730:730+400]))
    degradation_rates_test.append(extract_yr_degrad(y_test[i]))
    
mse = mean_squared_error(model_pred, degradation_rates_test)
np.sqrt(mse)

In [ ]:
# Plotting decay rate (predicted vs target)

plt.figure(figsize=(13,3))
plt.plot(degradation_rates_test[2:50], 'o')
plt.plot(model_pred[2:50], 'o')
plt.xlabel("Time series sample #")
plt.ylabel("Yearly decay rate")
plt.title("NN prediction vs actual yearly decay rate")
plt.legend(["Actual","NN predicted"])
plt.show()